# Phase 2: Model Development

## Ziel der Phase
In dieser Phase entwickeln wir ein Machine Learning Modell, das Fahrer in F2 und F3 nach ihrer Wahrscheinlichkeit für einen späteren Formel 1 Einstieg priorisiert.  
Der Fokus liegt auf einem realistischen Setup ohne Datenleckage und mit nachvollziehbarer Evaluation.

---

## Ausgangslage und Inputdaten

### Verwendeter Datensatz
Wir arbeiten mit einem konsolidierten Feature Datensatz aus F2 und F3, ergänzt um ein Label für einen späteren F1 Einstieg.

Datei  
`data/model_input/f2_f3_features_with_f1_label.csv`

### Wichtigste Spalten
- Identifikatoren: `driver_code`, `driver_name`, `team_name`, `series`, `year`
- Zielvariable: `f1_entry`  
  True bedeutet, dass der Fahrer später in F1 gestartet ist  
  False bedeutet, dass kein späterer F1 Start beobachtet wurde
- Performance Features: Punkte, Platzierungen, Konstanz, Ausfallquote, Pace, Rennerfahrung

---

## Methodische Entscheidungen

### Warum ein zeitbasierter Train Test Split
Da die Zielsetzung eine Prognose auf Basis von Performance bis Saison t ist, verwenden wir einen zeitbasierten Split.  
So simulieren wir einen realistischen Einsatz, bei dem nur vergangene Daten für Vorhersagen genutzt werden.

Beispiel Cutoff
- Training: 2017 bis 2021
- Test: 2022 bis 2023
- 2024 bis 2025 werden ausgeschlossen, weil für diese Jahre im Datensatz keine beobachtbaren F1 Einstiege enthalten sind und dadurch die Labels unvollständig sein können

---

## Baseline Modell

### Modellwahl
Als Baseline verwenden wir eine logistische Regression, weil sie
- Wahrscheinlichkeiten liefert
- gut interpretierbar ist
- bei kleinen Datensätzen stabil funktioniert
- als Referenz für spätere Modelle dient

### Pipeline Aufbau
Wir nutzen eine Pipeline, um Datenleckage zu vermeiden.

Bestandteile
1. Imputation mit Median  
   Missing Values werden mit dem Median aus dem Trainingsset ersetzt
2. Standardisierung  
   Features werden skaliert, damit Koeffizienten vergleichbar sind
3. Logistische Regression  
   `class_weight = balanced`, um das Ungleichgewicht der Klassen zu berücksichtigen

---

## Evaluation

### Warum ROC AUC und PR AUC
Da F1 Einstiege selten sind, ist Accuracy ungeeignet.  
Wir bewerten daher das Ranking der Wahrscheinlichkeiten mit
- ROC AUC: generelle Ranking Fähigkeit
- PR AUC: besonders aussagekräftig bei seltenen positiven Fällen

### Ergebnisinterpretation
Ein gutes Modell zeigt
- PR AUC deutlich über der Base Rate
- sinnvolles Ranking für Top k Fahrer

---

## Top k Ansatz für die Anwendung

### Motivation
Eine fixe Schwelle wie 0.5 ist bei seltenen Ereignissen oft ungeeignet.  
In einem realistischen Scouting oder Investment Setting interessiert, welche Fahrer relativ gesehen am stärksten sind.

### Vorgehen
Wir sortieren Fahrer nach vorhergesagter Wahrscheinlichkeit und wählen die Top k Prozent.  
Danach messen wir
- Precision in Top k  
  Trefferquote unter den ausgewählten Fahrern
- Recall in Top k  
  Anteil aller späteren F1 Fahrer, die in dieser Auswahl enthalten sind

### Visualisierung
Wir erstellen eine Recall k Prozent Kurve, um zu zeigen, wie stark das Modell positive Fälle in einer kleinen Teilmenge konzentriert.

---

## Interpretierbarkeit und Feature Einfluss

### Koeffizientenanalyse
Da Features standardisiert wurden, sind die Koeffizienten der logistischen Regression vergleichbar.

Interpretation
- Positiver Koeffizient erhöht die Wahrscheinlichkeit
- Negativer Koeffizient senkt die Wahrscheinlichkeit
- Betrag des Koeffizienten zeigt die Einflussstärke

Hinweis
Koeffizienten zeigen Zusammenhänge, keine Kausalität.  
Bei stark korrelierten Performance Features können Vorzeichen kontraintuitiv wirken, weil das Modell den relativen Informationsgewinn unter Kontrolle anderer Variablen abbildet.

---

## Modellvergleich als Validierungsschritt

### Motivation
Zur Einordnung der Baseline vergleichen wir die logistische Regression mit einem Random Forest Modell.

Ziel
- prüfen, ob nichtlineare Modelle signifikant besser sind
- Trade off zwischen Performance und Interpretierbarkeit diskutieren

Vergleichsmetriken
- ROC AUC
- PR AUC
- optional Top k Recall

---

## Limitationen

- Zeitverzögerte Labels: F1 Einstiege können erst Jahre später sichtbar werden
- Historische Verzerrungen: Karrierepfade ändern sich über Zeit
- Selektionsbias: Nur Fahrer mit dokumentierten Daten
- Feature Korrelationen: erschweren eindeutige Interpretation einzelner Effekte

---

## Output der Phase

Artefakte dieser Phase
- Baseline Notebook mit Modell, Evaluation, Top k und Interpretation  
  `notebooks/03_baseline_logistic_regression.ipynb`
- Modellvergleich Notebook  
  `notebooks/04_model_comparison_random_forest.ipynb`
- Optional gespeicherte Splits  
  `data/model_input/splits/`

Diese Phase liefert damit sowohl eine funktionierende Modellierung als auch nachvollziehbare Evaluation und Interpretierbarkeit.
